In [ ]:
from helper import*
import params as cfg

In [ ]:
#   Group-Task-Time
M11 = ['m11_1.zip','m11_2.zip','m11_3.zip','m11_4.zip','m11_5.zip']
M21 = ['m21_1.zip','m21_2.zip','m21_3.zip','m21_4.zip','m21_5.zip']
M12 = ['m12_1.zip','m12_2.zip','m12_3.zip','m12_4.zip','m12_5.zip']
M22 = ['m22_1.zip','m22_2.zip','m22_3.zip','m22_4.zip','m22_5.zip']
C11 = ['c11_1.zip','c11_2.zip','c11_3.zip','c11_4.zip','c11_5.zip']
C21 = ['c21_1.zip','c21_2.zip','c21_3.zip','c21_4.zip','c21_5.zip']
C12 = ['c12_1.zip','c12_2.zip','c12_3.zip','c12_4.zip','c12_5.zip']
C22 = ['c22_1.zip','c22_2.zip','c22_3.zip','c22_4.zip','c22_5.zip']
localDir = "/Users/joshuaighalo/Documents/datasets/eeg/v2/"
destDir = "/Users/joshuaighalo/Documents/datasets/eeg/v2/dest/"
os.system('rm -rf %s/*' % destDir)
fs_setting = 'constant'

In [ ]:
def multipleFilesNotch(input_3d,fs):
    notch_out = []
    filtering = filters()
    for i in range(input_3d.shape[0]):
        notch_out.append(filtering.notch(input_3d[i,:,:],fs))
    return np.array(notch_out)

def multipleFilesbutterBandpass(input_3d,lowcut,highcut,fs):
    butter_out = []
    filtering = filters()
    for i in range(input_3d.shape[0]):
        butter_out.append(filtering.butterBandPass(input_3d[i,:,:],lowcut,highcut,fs))
    return np.array(butter_out)

In [ ]:
#   Extract eeg data from participants within the groups
M11 = zipExtract(M11,localDir,destDir,"M11",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
M21 = zipExtract(M21,localDir,destDir,"M21",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
M12 = zipExtract(M12,localDir,destDir,"M12",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
M22 = zipExtract(M22,localDir,destDir,"M22",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C11 = zipExtract(C11,localDir,destDir,"C11",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C21 = zipExtract(C21,localDir,destDir,"C21",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C12 = zipExtract(C12,localDir,destDir,"C12",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C22 = zipExtract(C22,localDir,destDir,"C22",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
time_s = np.arange(0,cfg.collection_time,1/cfg.fs)

In [ ]:
#   Notch filters data collected from participants
noc_M11 = multipleFilesNotch(M11,cfg.fs)
noc_M21 = multipleFilesNotch(M21,cfg.fs)
noc_M12 = multipleFilesNotch(M12,cfg.fs)
noc_M22 = multipleFilesNotch(M22,cfg.fs)
noc_C11 = multipleFilesNotch(C11,cfg.fs)
noc_C21 = multipleFilesNotch(C21,cfg.fs)
noc_C12 = multipleFilesNotch(C12,cfg.fs)
noc_C22 = multipleFilesNotch(C22,cfg.fs)

In [ ]:
#   Bandpass filters data collected from participants
band_M11 = multipleFilesbutterBandpass(noc_M11,cfg.lowcut,cfg.highcut,cfg.fs)
band_M21 = multipleFilesbutterBandpass(noc_M21,cfg.lowcut,cfg.highcut,cfg.fs)
band_M12 = multipleFilesbutterBandpass(noc_M12,cfg.lowcut,cfg.highcut,cfg.fs)
band_M22 = multipleFilesbutterBandpass(noc_M22,cfg.lowcut,cfg.highcut,cfg.fs)
band_C11 = multipleFilesbutterBandpass(noc_C11,cfg.lowcut,cfg.highcut,cfg.fs)
band_C21 = multipleFilesbutterBandpass(noc_C21,cfg.lowcut,cfg.highcut,cfg.fs)
band_C12 = multipleFilesbutterBandpass(noc_C12,cfg.lowcut,cfg.highcut,cfg.fs)
band_C22 = multipleFilesbutterBandpass(noc_C22,cfg.lowcut,cfg.highcut,cfg.fs)

In [ ]:
def eemd_ica(input_1D,fs,show_plots=False):
    """
    Inputs:  input: 2D array of EEG data (samples x channels)
                fs: sampling frequency
    Outputs: restored signal (samples x channels)
    """

    def ICA(input,fs):
        """
        Inputs:  input: 2D array of EEG data (samples x channels)
                    fs: sampling frequency
        Outputs: restored signal (samples x channels)
        """

        def icaHighpass(data,cutoff,fs):
            def params_fnc(data,cutoff,fs,order=4):
                nyq = 0.5 * fs
                normal_cutoff = cutoff / nyq
                b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
                y = signal.filtfilt(b, a, data)
                return y
            filterEEG = []
            for i in range(len(data.T)):
                filterEEG.append(params_fnc(data.T[i],cutoff,fs))
            filterEEG = np.array(filterEEG).T
            return filterEEG

        def confidenceInterval(samples):
        #   At 95% significance level, tN -1 = 2.201
            means = np.mean(samples)
            std_dev = np.std(samples)
            standard_error = std_dev/np.sqrt(len(samples))
            lower_95_perc_bound = means - 2.201*standard_error
            upper_95_perc_bound = means + 2.201*standard_error
            return upper_95_perc_bound

        def setZeros(data,index):
            def params(data):
                return np.zeros(len(data))
            zeros = []
            for i in range(len(index)):
                zeros.append(params(data.T[index[i]]))
            zeros = np.array(zeros)
            return zeros

        hpEEG = icaHighpass(input,1,fs)
        ica_ = FastICA(n_components=len(input.T), random_state=0, tol=0.0001)
        comps = ica_.fit_transform(hpEEG)
        comps_kurtosis = kurtosis(comps)

        #   Computing CI on to set threshold
        threshold_kurt = confidenceInterval(comps_kurtosis)
        "compare threshold with extracted parameter values"
        #   Extract epochs
        bool_ArtfCompsKurt = [comps_kurtosis>threshold_kurt]
        idx_ArtfCompsKurt = np.asarray(np.where(bool_ArtfCompsKurt[0]==True))

        #   Merge index of components detected as artifacts by kurtosis, skewness, and sample entropy
        idx_artf_comps = idx_ArtfCompsKurt
        idx_artf_comps = np.unique(idx_artf_comps)

        "Component identified as artifact is converted to arrays of zeros"
        rejected_comps = setZeros(comps,idx_artf_comps)


        "Return zero-ed ICs into the original windows per ICs"
        for i in range(len(idx_artf_comps)):
            idx_rejected_comps = np.arange(len(rejected_comps))
            comps.T[idx_artf_comps[i]] = rejected_comps[idx_rejected_comps[i]]


        "Recover clean signal from clean ICs"
        restored = ica_.inverse_transform(comps)
        return restored

    # import empirical mode decomposition
    imf = emd.sift.sift(input_1D)

    # plot all columns in the imf array against time
    if show_plots:
        fig,ax = plt.subplots(4,3,figsize=(20,8))
        fig.subplots_adjust(hspace=1)
        for i in range(4):
            for j in range(3):
                ax[i,j].plot(time,imf[:,i*3+j])
                ax[i,j].set_title('IMF {}'.format(i*3+j+1))
                ax[i,j].set_xlabel('Time (ms)')
                ax[i,j].set_ylabel('Amplitude (uV)')
        plt.show()

    ica_imf = ICA(imf,fs)
    cleaned_signal = np.sum(ica_imf,axis=1)

    if show_plots:
        fig,ax = plt.subplots(2,1,figsize=(20,8))
        fig.subplots_adjust(hspace=0.4)
        ax[0].plot(time,czEEG,color='red')
        ax[0].set_title('Raw EEG Data')
        ax[0].set_ylabel('Amplitude (uV)')
        ax[1].plot(time,cleaned_signal,color='blue')
        ax[1].set_title('Cleaned EEG Data')
        ax[1].set_xlabel('Time (ms)')
        ax[1].set_ylabel('Amplitude (uV)')
        plt.show()

    return cleaned_signal